In [5]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [6]:
# Load libraries
import numpy as np
import pandas as pd
import sys
import itertools
import os
import pickle

from classes import functions as fcy


#--- Notebook run settings
run = 'command line' # 'manual' 'command line'
#run = 'manual'

# Settings for the multiple linear regression model (MLR)

#--- Arguments
if run == 'command line':
    
    # Read command line arguments
    arguments = os.environ['NB_ARGS']
    today_str = arguments
    print(today_str)

if run == 'manual':
    today_str = '230207_Full'
    
%matplotlib inline
%config Completer.use_jedi = False 

In [7]:
today_str

'230207_Full'

In [8]:
#----------------------#
#   Create dataframe   #
#----------------------#

#--- Climate influence on decomposition expressed by Temp and Moisture variables (except Humidity)
var_groups1 = {'temperature':['MAT','MATsoil'],
              
              'moisture':['MAP', 'Humidity', 'Soil_water', 'Soil_water_max'
                         ],
              
              'Substrate_quality':['PFT','CN_ratio', 'Wood_durability'
                                  ],
              
              'Size':['DBH_mean'], 
              #'Mortality':['Mortality'], # excluding mortality bc it is a non-signficant covariate
               
               # excluding management from pot. models bc it is a non signficant co-variate for its subset
              'Management':['Managed'] 
              }

    
#--- Climate influence expressed by CDI
var_groups2 = {'CDI':['CDI'],
              
              'Substrate_quality':['PFT','CN_ratio', 'Wood_durability'
                                  ],
              
              'Size':['DBH_mean'], 
              #'Mortality':['Mortality'], # excluding mortality bc it is a non-signficant covariate
               
              # excluding management from pot. models bc it is a non signficant co-variate for its subset
            'Management':['Managed']
             }




#--- Modell generation

# Removing moisture vars for the full data set (incl. management)
if 'Moisture' not in today_str:
    var_groups1['moisture'] = ['MAP', 'Humidity'] 

# Removing substrate vars for the full data set (incl. management)
if 'WoodQ' not in today_str:
    var_groups1['Substrate_quality'] = ['PFT'] 
    var_groups2['Substrate_quality'] = ['PFT']
    
    


#--- Make unique combinations of variables
# For seperate climate variables
if 'Management' not in today_str:
    var_groups1.pop('Management', None)
    var_groups1.pop('Management', None)
    # Set interaction to true to test interaction effect, here set to false to minimise runtime
    var_group1_models = fcy.unique_models_fixed_interaction(var_groups1, interaction=False) 
else:
    # Set interaction to true to test interaction effect, here set to false to minimise runtime
    var_group1_models = fcy.unique_models_fixed_interaction(var_groups1, interaction=False)
# For CDI
if 'Management' not in today_str:
    var_groups2.pop('Management', None)
    var_groups2.pop('Management', None)
    # Set interaction to true to test interaction effect, here set to false to minimise runtime
    var_group2_models = fcy.unique_models_fixed_interaction(var_groups2, interaction=False)
else: 
    # Set interaction to true to test interaction effect, here set to false to minimise runtime
    var_group2_models = fcy.unique_models_fixed_interaction(var_groups2, interaction=False)

# Combine to experiment list
var_models = var_group1_models + var_group2_models

# Remove duplicates
var_models = list(np.unique(np.array(var_models))) 

var_models = ['MTF ~ '+m for m in var_models]
print(len(var_models))

# Store experiments for regressions
with open('spe_reg_results/'+today_str+'_Spe_reg_experiments_list.pkl', 'wb') as f:
    pickle.dump(var_models, f)

39


In [9]:
import pickle
with open('spe_reg_results/'+today_str+'_Spe_reg_experiments_list.pkl', 'rb') as f:
    experiments = pickle.load(f)

display(experiments) 

['MTF ~ CDI',
 'MTF ~ CDI + DBH_mean',
 'MTF ~ CDI + PFT',
 'MTF ~ CDI + PFT + DBH_mean',
 'MTF ~ DBH_mean',
 'MTF ~ Humidity',
 'MTF ~ Humidity + DBH_mean',
 'MTF ~ Humidity + PFT',
 'MTF ~ Humidity + PFT + DBH_mean',
 'MTF ~ MAP',
 'MTF ~ MAP + DBH_mean',
 'MTF ~ MAP + PFT',
 'MTF ~ MAP + PFT + DBH_mean',
 'MTF ~ MAT',
 'MTF ~ MAT + DBH_mean',
 'MTF ~ MAT + Humidity',
 'MTF ~ MAT + Humidity + DBH_mean',
 'MTF ~ MAT + Humidity + PFT',
 'MTF ~ MAT + Humidity + PFT + DBH_mean',
 'MTF ~ MAT + MAP',
 'MTF ~ MAT + MAP + DBH_mean',
 'MTF ~ MAT + MAP + PFT',
 'MTF ~ MAT + MAP + PFT + DBH_mean',
 'MTF ~ MAT + PFT',
 'MTF ~ MAT + PFT + DBH_mean',
 'MTF ~ MATsoil',
 'MTF ~ MATsoil + DBH_mean',
 'MTF ~ MATsoil + Humidity',
 'MTF ~ MATsoil + Humidity + DBH_mean',
 'MTF ~ MATsoil + Humidity + PFT',
 'MTF ~ MATsoil + Humidity + PFT + DBH_mean',
 'MTF ~ MATsoil + MAP',
 'MTF ~ MATsoil + MAP + DBH_mean',
 'MTF ~ MATsoil + MAP + PFT',
 'MTF ~ MATsoil + MAP + PFT + DBH_mean',
 'MTF ~ MATsoil + PFT',
 '